# Module: Fetch Data
*get data from TSETMC and directly write to db*

## Prepearing Classification Data

In [1]:
from typing import Union
import pandas as pd
import requests
from datetime import datetime
import pytse_client as tse
import fetch
from config import *
from utils import debug_log, data_log, ar_to_fa_series, ar_to_fa, fa_to_ar, fa_to_ar_series, flatten_json

In [2]:
pd.set_option('display.max_columns', 30)

## Definition & Assignment

In [3]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Referer": "http://main.tsetmc.com/StaticContent/WebServiceHelp",
}
cookie_jar = {"ASP.NET_SessionId": "wa40en1alwxzjnqehjntrv5j"}
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']

currentDateTime = datetime.now().strftime("%Y-%m-%d %H-%M-%S %p")
currentDateTime

In [4]:
last_date = fetch.last_possible_deven()
instruments, shares = fetch.InstrumentAndShare(0, 0)

shares.to_csv(f"./database/shares.csv")
instruments.to_csv(f"./database/instruments.csv")

print(last_date)

20230909


## Update Identity

In [5]:
fetch.update_identities()

HTTPError: 500 Server Error: Internal Server Error for url: http://cdn.tsetmc.com/api/Instrument/GetInstrumentIdentity/30106839080444358

## Getting Prices

In [6]:
daily_prices = fetch.get_daily_prices(tickers)
len(daily_prices.keys())

2172

## Categorization

In [5]:
identities = pd.read_csv('./database/identities.csv', index_col='insCode', dtype={'insCode':str})

In [6]:
def filter_tickers_by_industry(cSecVal: int) -> list:
    return list(identities[identities['sector_cSecVal'] == cSecVal].index)

In [7]:
khodroi = filter_tickers_by_industry(34)

## Volume Filter

In [19]:

def volume_filter(insCodes: list) -> list:
    
    daily_prices = fetch.get_daily_prices(insCodes)
    
    suspicious = []
    ticker_names= []
    
    for insCode in insCodes:
        debug_log.info(f'Going for insCode: {insCode}')
        first_row = daily_prices[insCode].iloc[0]
        if ((first_row['<VOL>'] > first_row['<VOL> MA 3']) and 
            (first_row['<VOL>'] > first_row['<VOL> MA 5']) and
            (first_row['<VOL>'] > first_row['<VOL> MA 9']) and
            (first_row['<VOL>'] > first_row['<VOL> MA 26']) and
            (first_row['<VOL>'] > first_row['<VOL> MA 52'])): # فعلا شرط ۹۰ روزه رو نمیذارم بخاطر شرایط کیری بازار ک اصلا هیچی معامله نمیشه
            suspicious.append(daily_prices[insCode])
            ticker_names.append(insCode)
            debug_log.info(f'{insCode} Checked.')
    print(len(suspicious),'\n', ticker_names, '\n')
    return suspicious


In [20]:
volume_filter(list(cleaned_data.keys()))

0 
 [] 



[]